In [40]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict, List, Annotated
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv, find_dotenv
import os
from langgraph.graph import add_messages
env_path = find_dotenv()
load_dotenv(env_path)

True

In [41]:
model = init_chat_model("google_genai:gemini-2.5-flash", api_key = os.getenv("GOOGLE_API_KEY"))

In [42]:
model.invoke("What is the radius of earth?")

AIMessage(content='The Earth is not a perfect sphere, so its radius varies slightly. However, the most commonly cited average radius is:\n\n*   **6,371 kilometers**\n*   **3,959 miles**\n\nTo be more precise:\n*   **Equatorial radius:** Approximately 6,378 km (3,963 miles)\n*   **Polar radius:** Approximately 6,357 km (3,950 miles)\n\nWhen people ask for "the radius of Earth," they usually refer to the average or mean radius of 6,371 km.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--9a3e2987-9454-4de2-9aa9-de0ed3188e1f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 449, 'total_tokens': 457, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 321}})

# STATE 

In [43]:
class JokeState(TypedDict):
    topic: str
    jokes: str
    explanation: str

# NODES

In [44]:
def generate_joke(state: JokeState):
    prompt = f"Generate a joke about {state['topic']}"
    joke = model.invoke(prompt).content
    return {"jokes": joke}

In [45]:
def explain_joke(state: JokeState):
    prompt = f"Explain the following joke: {state['jokes']}"
    explanation = model.invoke(prompt).content
    return {"explanation": explanation}

# Chekpointers

In [46]:
checkpoint = InMemorySaver()

# GRAPH

In [47]:
graph = StateGraph(JokeState)
graph.add_node("generate joke", generate_joke)
graph.add_node("explain joke", explain_joke)

graph.add_edge(START, "generate joke")
graph.add_edge("generate joke", "explain joke")
graph.add_edge("explain joke", END)
workflow = graph.compile(checkpointer=checkpoint)

# Threads for different convos

In [48]:
config1 = {
    "configurable": {
        "thread_id": "1"
    }
}

In [49]:
workflow.invoke({"topic": "computers"}, config=config1)

{'topic': 'computers',
 'jokes': 'Why was the computer cold?\n\nBecause it left its Windows open!',
 'explanation': 'This is a classic pun! Here\'s why it\'s funny:\n\n1.  **"Windows" (Computer Meaning):** In the context of a computer, "Windows" refers to the popular **Microsoft Windows operating system**. When you\'re using a computer, you open "windows" to run applications, browse the internet, etc.\n\n2.  **"Windows" (Literal Meaning):** In the context of a house or building, "windows" are the glass panes that let light in and can be opened to let fresh air in or to cool down a room.\n\n3.  **The Pun:** The joke plays on the double meaning of "Windows."\n    *   **If a person leaves their *physical windows* open in cold weather, they (and their house) will get cold.**\n    *   The joke pretends that the computer, like a person, can "feel" cold because it "left its Windows open" – referring to the operating system, not actual physical windows.\n\nThe humor comes from the silly image 

In [50]:
workflow.get_state(config=config1)

StateSnapshot(values={'topic': 'computers', 'jokes': 'Why was the computer cold?\n\nBecause it left its Windows open!', 'explanation': 'This is a classic pun! Here\'s why it\'s funny:\n\n1.  **"Windows" (Computer Meaning):** In the context of a computer, "Windows" refers to the popular **Microsoft Windows operating system**. When you\'re using a computer, you open "windows" to run applications, browse the internet, etc.\n\n2.  **"Windows" (Literal Meaning):** In the context of a house or building, "windows" are the glass panes that let light in and can be opened to let fresh air in or to cool down a room.\n\n3.  **The Pun:** The joke plays on the double meaning of "Windows."\n    *   **If a person leaves their *physical windows* open in cold weather, they (and their house) will get cold.**\n    *   The joke pretends that the computer, like a person, can "feel" cold because it "left its Windows open" – referring to the operating system, not actual physical windows.\n\nThe humor comes fr

In [51]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'computers', 'jokes': 'Why was the computer cold?\n\nBecause it left its Windows open!', 'explanation': 'This is a classic pun! Here\'s why it\'s funny:\n\n1.  **"Windows" (Computer Meaning):** In the context of a computer, "Windows" refers to the popular **Microsoft Windows operating system**. When you\'re using a computer, you open "windows" to run applications, browse the internet, etc.\n\n2.  **"Windows" (Literal Meaning):** In the context of a house or building, "windows" are the glass panes that let light in and can be opened to let fresh air in or to cool down a room.\n\n3.  **The Pun:** The joke plays on the double meaning of "Windows."\n    *   **If a person leaves their *physical windows* open in cold weather, they (and their house) will get cold.**\n    *   The joke pretends that the computer, like a person, can "feel" cold because it "left its Windows open" – referring to the operating system, not actual physical windows.\n\nThe humor comes f

In [58]:
config2 = {
    'configurable': {
        'thread_id': '2'
    }
}

In [59]:
workflow.invoke({"topic": "lotion"}, config=config2)

{'topic': 'lotion',
 'jokes': 'Why did the lotion break up with the moisturizer?\n\nIt felt like they were always **rubbing** each other the wrong way!',
 'explanation': 'This joke is a pun that plays on the word "rubbing" and the common idiom "rubbing someone the wrong way."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning:** Lotion and moisturizer are both products that you **rub** into your skin. So, literally, they are constantly involved in the act of "rubbing."\n\n2.  **Idiomatic Meaning:** The phrase "**rubbing someone the wrong way**" is an idiom that means to annoy, irritate, or displease someone. When people are "rubbing each other the wrong way," it means they are constantly clashing, disagreeing, or causing friction.\n\nThe humor comes from the fact that while lotion and moisturizer are *literally* "rubbing" substances, the joke uses the idiom to explain their relationship problems. They broke up because, metaphorically, they were constantly irritating each other, just li

In [60]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'lotion', 'jokes': 'Why did the lotion break up with the moisturizer?\n\nIt felt like they were always **rubbing** each other the wrong way!', 'explanation': 'This joke is a pun that plays on the word "rubbing" and the common idiom "rubbing someone the wrong way."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning:** Lotion and moisturizer are both products that you **rub** into your skin. So, literally, they are constantly involved in the act of "rubbing."\n\n2.  **Idiomatic Meaning:** The phrase "**rubbing someone the wrong way**" is an idiom that means to annoy, irritate, or displease someone. When people are "rubbing each other the wrong way," it means they are constantly clashing, disagreeing, or causing friction.\n\nThe humor comes from the fact that while lotion and moisturizer are *literally* "rubbing" substances, the joke uses the idiom to explain their relationship problems. They broke up because, metaphorically, they were constantly irritating 

In [61]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'lotion', 'jokes': 'Why did the lotion break up with the moisturizer?\n\nIt felt like they were always **rubbing** each other the wrong way!', 'explanation': 'This joke is a pun that plays on the word "rubbing" and the common idiom "rubbing someone the wrong way."\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning:** Lotion and moisturizer are both products that you **rub** into your skin. So, literally, they are constantly involved in the act of "rubbing."\n\n2.  **Idiomatic Meaning:** The phrase "**rubbing someone the wrong way**" is an idiom that means to annoy, irritate, or displease someone. When people are "rubbing each other the wrong way," it means they are constantly clashing, disagreeing, or causing friction.\n\nThe humor comes from the fact that while lotion and moisturizer are *literally* "rubbing" substances, the joke uses the idiom to explain their relationship problems. They broke up because, metaphorically, they were constantly irritating

# Chekpoint id to time travel

In [62]:
workflow.invoke({"topic": "computers"}, {'configurable': {'thread_id': '2'}, 'checkpoint_id':'1f0dff0d-840e-6ac2-8000-b68d3c6335eb'})

{'topic': 'computers',
 'jokes': 'Why did the computer go to the doctor?\n\nBecause it had a virus!',
 'explanation': 'This joke is a classic example of a **pun** or a **play on words**, specifically using the word "virus."\n\nHere\'s the breakdown:\n\n1.  **The Human Context:** When a *person* "has a virus," it means they are sick with a biological infection (like the flu, a cold, etc.). The natural and logical thing to do in this situation is to "go to the doctor" to get help or treatment.\n\n2.  **The Computer Context:** When a *computer* "has a virus," it doesn\'t mean it\'s biologically sick. Instead, it means it\'s infected with a malicious piece of software designed to interfere with its operation, corrupt data, or cause other problems.\n\n3.  **The Pun:** The joke uses the word "virus" in both its biological (human illness) and technological (computer malware) senses. The humor comes from applying the human solution (going to the doctor) to the computer\'s problem, even though 

In [65]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'computers', 'jokes': 'Why did the computer go to the doctor?\n\nBecause it had a virus!', 'explanation': 'This joke is a classic example of a **pun** or a **play on words**, specifically using the word "virus."\n\nHere\'s the breakdown:\n\n1.  **The Human Context:** When a *person* "has a virus," it means they are sick with a biological infection (like the flu, a cold, etc.). The natural and logical thing to do in this situation is to "go to the doctor" to get help or treatment.\n\n2.  **The Computer Context:** When a *computer* "has a virus," it doesn\'t mean it\'s biologically sick. Instead, it means it\'s infected with a malicious piece of software designed to interfere with its operation, corrupt data, or cause other problems.\n\n3.  **The Pun:** The joke uses the word "virus" in both its biological (human illness) and technological (computer malware) senses. The humor comes from applying the human solution (going to the doctor) to the computer\'s p